### <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    

```


```python
def get_data(args1, *args):
    ....
    ....
    return train_loader, test_loader
```

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt  # one of the best graphics library for python
plt.style.use('ggplot')

In [2]:
import os
import random

import pandas as pd
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, transforms, models

import time

from typing import Iterable
from dataclasses import dataclass


from torch.optim import lr_scheduler


In [3]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter(log_dir='TFlearning_logs')

In [4]:
class KenyanFood13Dataset(Dataset):
    """
    Custom Dataset for loading images and labels for the KenyanFood13 challenge.
    This class reads a CSV file with columns 'id' and 'class', and loads images stored as '<id>.jpg'
    in the specified image root directory. It also supports optional image resizing and transformations.
    The train flag is used to split the full dataset into 80% training and 20% validation.
    """
    
    def __init__(self, csv_file, image_root, train=True, image_shape=None, transform=None):
        """
        Initialization method.
        
        Parameters:
        
        csv_file (str): Path to the CSV file containing image ids and class labels.
        
        image_root (str): Directory where image files are stored. Images are expected to be named as '<id>.jpg'.
        
        train (bool): If True, returns 80% of the data for training; if False, returns 20% for validation.
        
        image_shape (int or tuple or list): [optional] If provided, image will be resized to the given shape.
                                            If an integer is provided, images are resized to (image_shape, image_shape).
                                            If a tuple/list is provided with one value, it is converted to square dimensions;
                                            otherwise, the tuple is used as is.
                                            
        transform (callable): Transformation function to be applied on the PIL image.
        """
        # Read the CSV file containing the image ids and their class labels
        self.data = pd.read_csv(csv_file)
        self.image_root = image_root
        self.train = train
        self.transform = transform
        
        # Process image_shape parameter
        if image_shape is not None:
            if isinstance(image_shape, int):
                self.image_shape = (image_shape, image_shape)
            elif isinstance(image_shape, (tuple, list)):
                assert len(image_shape) in [1, 2], 'Invalid image_shape tuple/list size'
                self.image_shape = (image_shape[0], image_shape[0]) if len(image_shape) == 1 else image_shape
            else:
                raise NotImplementedError("image_shape must be an int, tuple, or list")
        else:
            self.image_shape = None
        
        # Create a mapping from class name to integer label
        self.classes = sorted(self.data['class'].unique())
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        
        # Build the list of samples (each sample is a tuple of image path and numeric label)
        full_samples = []
        for _, row in self.data.iterrows():
            img_id = row['id']
            label_str = row['class']
            label = self.class_to_idx[label_str]
            img_path = os.path.join(self.image_root, f"{img_id}.jpg")
            full_samples.append((img_path, label))
        
        # Shuffle the full sample list with a fixed seed for reproducibility and split into train/validation.
        random.seed(42)
        random.shuffle(full_samples)
        n_train = int(0.8 * len(full_samples))
        if self.train:
            self.samples = full_samples[:n_train]
        else:
            self.samples = full_samples[n_train:]
    
    def __len__(self):
        """
        Returns:
            int: Total number of samples in the dataset.
        """
        return len(self.samples)
    
    def __getitem__(self, idx):
        """
        For the given index, returns the processed image and its label.
        
        Parameters:
            idx (int): Index of the sample to retrieve.
            
        Returns:
            image: Processed image.
            target (int): Numeric label for the image.
        """
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Resize image if image_shape is specified
        if self.image_shape is not None:
            image = TF.resize(image, self.image_shape)
            
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    
    def class_name(self, label):
        """
        Gets the class name corresponding to a numeric label.
        
        Parameters:
            label (int): Numeric label.
        
        Returns:
            str: Class name.
        """
        return self.classes[label]
    
    

In [5]:
def image_preprocess_transforms():
    
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
        ])
    
    return preprocess

def image_common_transforms(mean=(0.4611, 0.4359, 0.3905), std=(0.2193, 0.2150, 0.2109)):
    preprocess = image_preprocess_transforms()
    
    common_transforms = transforms.Compose([
        preprocess,
        transforms.Normalize(mean, std)
    ])
    
    return common_transforms

def get_mean_std():
    
    mean = [0.485, 0.456, 0.406] 
    std = [0.229, 0.224, 0.225]
    
    return mean, std

def data_loader(dataset, batch_size=16, shuffle=False, num_workers=2): 
    loader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         shuffle=shuffle)
    
    return loader

# Define the missing data_augmentaaion_preprocess function
def data_augmentation_preprocess(mean, std):
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

In [6]:
def get_data(batch_size, data_root, num_workers=4, data_augmentation=True, image_shape=None, csv_file=None):
    """
    Creates training and validation DataLoaders for the KenyanFood13 dataset.
    
    Parameters:
        batch_size (int): Number of samples per batch.
        data_root (str): Root directory containing your dataset.
            Expected structure:
                data_root/
                    training/
                        images/ (training images)
                        labels.csv (training labels)
                    validation/
                        images/ (validation images)
                        labels.csv (validation labels)
        num_workers (int): Number of worker threads for DataLoader.
        data_augmentation (bool): Apply data augmentation on training data if True.
        image_shape (int, tuple, or list): Desired image size.
        csv_file (str, optional): If provided, use this as CSV file path for training;
                                  otherwise, assumes it is located under the respective folder.
    
    Returns:
        tuple: (train_loader, validation_loader)
    """
    # Define paths for training images
    train_csv = csv_file or os.path.join(data_root, 'train.csv')
    # Assume training images are stored in a folder named 'images'
    train_image_root = os.path.join(data_root, 'images')
    
    # Get normalization parameters and common transformations
    mean, std = get_mean_std()  # adjust if your get_mean_std needs additional arguments
    common_transforms = image_common_transforms(mean, std)
    
    if data_augmentation:
        train_transforms = data_augmentation_preprocess(mean, std)
    else:
        train_transforms = common_transforms

    # Create the training and validation datasets using the KenyanFood13Dataset class
    train_dataset = KenyanFood13Dataset(train_csv, train_image_root, train=True, image_shape=image_shape, transform=train_transforms)
    validation_dataset = KenyanFood13Dataset(train_csv, train_image_root, train=False, image_shape=image_shape, transform=common_transforms)
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_loader, validation_loader

## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/opencv-pytorch-project-2-classification-round-3" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [7]:
@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 21  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

In [8]:
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 32  # amount of data to pass through the network at each forward-backward iteration  
    epochs_count: int = 10 
    init_learning_rate: float = 0.001  # initial learning rate for lr scheduler
    decay_rate: float = 0.1  
    log_interval: int = 1  # how many batches to wait before logging training status  
    test_interval: int = 1  
    data_root: str = './opencv-pytorch-project-2-classification-round-3/'
    csv_file: str = './opencv-pytorch-project-2-classification-round-3/train.csv'
    num_workers: int = 2  # number of concurrent processes using to prepare data  
    device: str = 'cuda'  
    lr_scheduler_patience: int = 5     # Patience (in epochs) for ReduceLROnPlateau scheduler
    lr_scheduler_factor: float = 0.3   # LR reduction factor when plateauing
    early_stopping_patience: int = 5  # Patience (in epochs) for early stopping
    



In [9]:
def setup_system(system_config: SystemConfiguration) -> None:
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

In [10]:
# food_classes = ['Ugali', 'Chapati', 'Mukimo', 'Matoke', 'Kachumbari', 'Sukuma Wiki', 'Nyama Choma', 'Githeri', 'Mandazi', 'Pilau', 'Kebab', 'Samaki', 'Wali']
dataset = KenyanFood13Dataset(TrainingConfiguration.csv_file, TrainingConfiguration.data_root, train=False, image_shape=224)
food_classes = dataset.classes
food_classes

['bhaji',
 'chapati',
 'githeri',
 'kachumbari',
 'kukuchoma',
 'mandazi',
 'masalachips',
 'matoke',
 'mukimo',
 'nyamachoma',
 'pilau',
 'sukumawiki',
 'ugali']

## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

In [11]:
def prediction(model, device, batch_input, max_prob=True):
    """
    get prediction for batch inputs
    """
    
    # send model to cpu/cuda according to your system configuration
    model.to(device)
    
    # it is important to do model.eval() before prediction
    model.eval()

    data = batch_input.to(device)

    output = model(data)

    # get probability score using softmax
    prob = F.softmax(output, dim=1)
    
    if max_prob:
        # get the max probability
        pred_prob = prob.data.max(dim=1)[0]
    else:
        pred_prob = prob.data
    
    # get the index of the max probability
    pred_index = prob.data.max(dim=1)[1]
    
    return pred_index.cpu().numpy(), pred_prob.cpu().numpy()

In [12]:
def get_target_and_prob(model, dataloader, device):
    """
    get targets and prediction probabilities
    """
    
    pred_prob = []
    targets = []
    
    for _, (data, target) in enumerate(dataloader):
        
        _, prob = prediction(model, device, data, max_prob=False)
        
        pred_prob.append(prob)
        
        target = target.numpy()
        targets.append(target)
        
    targets = np.concatenate(targets)
    targets = targets.astype(int)
    pred_prob = np.concatenate(pred_prob, axis=0)
    
    return targets, pred_prob
    
    

## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [13]:
def train(
    train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader, epoch_idx: int
) -> None:
    
    # change model in training mood
    model.train()
    
    # to get batch loss
    batch_loss = np.array([])
    
    # to get batch accuracy
    batch_acc = np.array([])
        
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # clone target
        indx_target = target.clone()
        # send data to device (its is medatory if GPU has to be used)
        data = data.to(train_config.device)
        # send target to device
        target = target.to(train_config.device)

        # reset parameters gradient to zero
        optimizer.zero_grad()
        
        # forward pass to the model
        output = model(data)
        
        # cross entropy loss
        loss = F.cross_entropy(output, target)
        
        # find gradients w.r.t training parameters
        loss.backward()
        # Update parameters using gardients
        optimizer.step()
        
        batch_loss = np.append(batch_loss, [loss.item()])
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
            
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]  
                        
        # correct prediction
        correct = pred.cpu().eq(indx_target).sum()
            
        # accuracy
        acc = float(correct) / float(len(data))
        
        batch_acc = np.append(batch_acc, [acc])
            
    epoch_loss = batch_loss.mean()
    epoch_acc = batch_acc.mean()
    print('Epoch: {} \nTrain Loss: {:.6f} Acc: {:.4f}'.format(epoch_idx, epoch_loss, epoch_acc))
    return epoch_loss, epoch_acc

In [14]:
import torch.nn.functional as F
from tqdm import tqdm

def validate(
    train_config: TrainingConfiguration,
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader,
) -> float:
    model.eval()
    test_loss = 0
    count_correct_predictions = 0
    # Iterate over test_loader with a tqdm progress bar.
    for data, target in tqdm(test_loader, desc="Validation", leave=False):
        indx_target = target.clone()
        data = data.to(train_config.device)
        target = target.to(train_config.device)
        print("Data device:", data.device)
        print("Target device:", target.device)
        
        with torch.no_grad():
            output = model(data)
        
        # Accumulate loss for each mini batch
        test_loss += F.cross_entropy(output, target).item()
        
        # Get predictions from probabilities
        prob = F.softmax(output, dim=1)
        pred = prob.data.max(dim=1)[1]
        
        count_correct_predictions += pred.cpu().eq(indx_target).sum().item()

    # Average loss over the number of mini-batches
    test_loss = test_loss / len(test_loader)
    
    # Compute accuracy over the entire dataset
    accuracy = 100. * count_correct_predictions / len(test_loader.dataset)
    
    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, count_correct_predictions, len(test_loader.dataset), accuracy
        )
    )
    return test_loss, accuracy / 100.0

In [15]:
def save_model(model, device, model_dir='models', model_file_name='final_kenyaFood_classifier.pt'):
    
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)

    # make sure you transfer the model to cpu. 
    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    
    if device == 'cuda':
        model.to('cuda')
    
    return model_path                                 

def load_model(model, model_dir='models', model_file_name='final_kenyaFood_classifier.pt'):
    model_path = os.path.join(model_dir, model_file_name)

    # loading the model and getting model parameters by using load_state_dict
    model.load_state_dict(torch.load(model_path))
    
    return model                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [16]:
def main(model, optimizer, scheduler=None, early_stopping=None, 
         system_configuration=SystemConfiguration(), 
         training_configuration=TrainingConfiguration(), 
         data_augmentation=True):
    
    print("Step 1: Setting up system configuration...")
    setup_system(system_configuration)

    print("Step 2: Initializing training parameters (batch size, num_workers, epochs)...")
    batch_size_to_set = training_configuration.batch_size
    num_workers_to_set = training_configuration.num_workers
    epoch_num_to_set = training_configuration.epochs_count

    print("Step 3: Selecting device...")
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        batch_size_to_set = 16
        num_workers_to_set = 4
    print("  Device selected:", device)

    # Move model to the selected device
    model.to(device)
    print("Model device:", next(model.parameters()).device)

    print("Step 4: Creating data loaders...")
    train_loader, test_loader = get_data(
        batch_size=batch_size_to_set,
        data_root=training_configuration.data_root,
        num_workers=num_workers_to_set,
        data_augmentation=data_augmentation
    )
    print("Data loaders created.")
    

    print("Step 5: Updating training configuration with system-adjusted parameters...")
    training_configuration = TrainingConfiguration(
        device=device,
        batch_size=batch_size_to_set,
        num_workers=num_workers_to_set
    )
        
    print("Step 6: Sending model to device...")
    model.to(training_configuration.device)
    print("Model device after update:", next(model.parameters()).device)

    best_loss = float('inf')
    
    # Arrays to track epoch train/test loss and accuracy
    epoch_train_loss = np.array([])
    epoch_test_loss = np.array([])
    epoch_train_acc = np.array([])
    epoch_test_acc = np.array([])
    
    print("Step 7: Performing initial validation...")
    init_val_loss, init_val_accuracy = validate(training_configuration, model, test_loader)
    print("Initial Test Loss: {:.6f}, Initial Test Accuracy: {:.3f}%".format(
        init_val_loss, init_val_accuracy * 100))
    
    t_begin = time.time()
    print("Step 8: Starting training loop for {} epochs...".format(training_configuration.epochs_count))
    for epoch in range(training_configuration.epochs_count):
        print("\nEpoch [{}/{}] Start".format(epoch + 1, training_configuration.epochs_count))
        
        # Train
        train_loss, train_acc = train(training_configuration, model, optimizer, train_loader, epoch)
        epoch_train_loss = np.append(epoch_train_loss, train_loss)
        epoch_train_acc = np.append(epoch_train_acc, train_acc)
        
        elapsed_time = time.time() - t_begin
        speed_epoch = elapsed_time / (epoch + 1)
        speed_batch = speed_epoch / len(train_loader)
        eta = speed_epoch * training_configuration.epochs_count - elapsed_time
        print("Elapsed {:.2f}s, {:.2f} s/epoch, {:.2f} s/batch, ETA {:.2f}s".format(
            elapsed_time, speed_epoch, speed_batch, eta))

        # Validate at set interval
        if epoch % training_configuration.test_interval == 0:
            print("Step 9: Running validation for epoch {}...".format(epoch + 1))
            current_loss, current_accuracy = validate(training_configuration, model, test_loader)
            epoch_test_loss = np.append(epoch_test_loss, current_loss)
            epoch_test_acc = np.append(epoch_test_acc, current_accuracy)
            
            # Early stopping check using validation loss
            if early_stopping is not None:
                early_stopping(current_loss, model)
                if early_stopping.early_stop:
                    print("Early stopping triggered")
                    break

            # Scheduler step/update; prefer validation loss for ReduceLROnPlateau
            if scheduler is not None:
                if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(current_loss)
                    print("Bad Epochs: {}".format(scheduler.num_bad_epochs))
                    print("Last LR = {}".format(scheduler._last_lr))
                else:
                    scheduler.step()

            # Save the model if loss has improved
            if current_loss < best_loss:
                best_loss = current_loss
                print("Model Improved. Saving the Model...\n")
                save_model(model, device=training_configuration.device)
        
    print("\nStep 10: Training complete.")
    print("Total time: {:.2f}s, Best Loss: {:.3f}".format(time.time() - t_begin, best_loss), flush=True)
    return model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc

## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [17]:
def pretrained_resnet18(transfer_learning=True, num_class=13):
    resnet = models.resnet18(pretrained=True)
    
    if transfer_learning:
        for param in resnet.parameters():
            param.requires_grad = False
            
    last_layer_in = resnet.fc.in_features
    resnet.fc = nn.Linear(last_layer_in, num_class)
    
    return resnet

In [18]:
class MyCNN(nn.Module):
    def __init__(self, num_classes=13):
        super(MyCNN, self).__init__()
        self.features = nn.Sequential(
            # Block 1: Input: 3x224x224 -> after conv: 32x224x224, then pool: 32x112x112
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            # Block 2: 32x112x112 -> 64x112x112, then pool: 64x56x56
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            # Block 3: 64x56x56 -> 128x56x56, then pool: 128x28x28
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        # Adaptive pooling to collapse spatial dimensions
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
          
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(128, 512),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

In [19]:
def plot_loss_accuracy(train_loss, val_loss, train_acc, val_acc, colors, 
                       loss_legend_loc='upper center', acc_legend_loc='upper left', 
                       fig_size=(20, 10), sub_plot1=(1, 2, 1), sub_plot2=(1, 2, 2)):
    
    plt.rcParams["figure.figsize"] = fig_size
    fig = plt.figure()
    
    plt.subplot(sub_plot1[0], sub_plot1[1], sub_plot1[2])
    
    for i in range(len(train_loss)):
        x_train = range(len(train_loss[i]))
        x_val = range(len(val_loss[i]))
        
        min_train_loss = train_loss[i].min()
        
        min_val_loss = val_loss[i].min()
        
        plt.plot(x_train, train_loss[i], linestyle='-', color='tab:{}'.format(colors[i]), 
                 label="TRAIN LOSS ({0:.4})".format(min_train_loss))
        plt.plot(x_val, val_loss[i], linestyle='--' , color='tab:{}'.format(colors[i]), 
                 label="VALID LOSS ({0:.4})".format(min_val_loss))
        
    plt.xlabel('epoch no.')
    plt.ylabel('loss')
    plt.legend(loc=loss_legend_loc)
    plt.title('Training and Validation Loss')
        
    plt.subplot(sub_plot2[0], sub_plot2[1], sub_plot2[2])
    
    for i in range(len(train_acc)):
        x_train = range(len(train_acc[i]))
        x_val = range(len(val_acc[i]))
        
        max_train_acc = train_acc[i].max() 
        
        max_val_acc = val_acc[i].max() 
        
        plt.plot(x_train, train_acc[i], linestyle='-', color='tab:{}'.format(colors[i]), 
                 label="TRAIN ACC ({0:.4})".format(max_train_acc))
        plt.plot(x_val, val_acc[i], linestyle='--' , color='tab:{}'.format(colors[i]), 
                 label="VALID ACC ({0:.4})".format(max_val_acc))
        
    plt.xlabel('epoch no.')
    plt.ylabel('accuracy')
    plt.legend(loc=acc_legend_loc)
    plt.title('Training and Validation Accuracy')
    
    fig.savefig('sample_loss_acc_plot.png')
    plt.show()
    
    return   

In [20]:
# Define early stopping class
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, checkpoint_path='checkpoint.pt'):
        """
        Args:
            patience (int): Number of epochs with no improvement after which training will be stopped.
            verbose (bool): If True, prints messages when the validation loss decreases.
            delta (float): Minimum change in the validation loss to qualify as an improvement.
            checkpoint_path (str): Path to save the best model.
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.checkpoint_path = checkpoint_path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss  # lower loss is better

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases."""
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...")
        torch.save(model.state_dict(), self.checkpoint_path)
        self.val_loss_min = val_loss

## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

In [21]:
# model = pretrained_resnet18()
model = MyCNN()

print(model)

train_config = TrainingConfiguration()

# optimizer (using Adam)
optimizer = optim.Adam(
    model.parameters(),
    lr=train_config.init_learning_rate
)

# Initialize early stopping
early_stopping = EarlyStopping(patience=train_config.early_stopping_patience, verbose=True)

# ReduceLROnPlateau scheduler to decay LR when validation stops improving
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                           factor=train_config.lr_scheduler_factor,
                                           patience=train_config.lr_scheduler_patience,
                                           verbose=True)

MyCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dr

C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
model, train_loss, train_acc, val_loss, val_acc = main(model, optimizer, 
                                                       scheduler=scheduler, 
                                                       early_stopping=early_stopping,
                                                       data_augmentation=False)

Step 1: Setting up system configuration...
Step 2: Initializing training parameters (batch size, num_workers, epochs)...
Step 3: Selecting device...
  Device selected: cuda
Model device: cuda:0
Step 4: Creating data loaders...
Data loaders created.
Step 5: Updating training configuration with system-adjusted parameters...
Step 6: Sending model to device...
Model device after update: cuda:0
Step 7: Performing initial validation...


Validation:   0%|          | 0/41 [00:00<?, ?it/s]

: 

## <font style="color:green">8. TensorBoard Log Link [5 Points]</font>

**Share your TensorBoard scalars logs link here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


Note: In light of the recent shutdown of tensorboard.dev, we have updated the submission requirements for your project. Instead of sharing a tensorboard.dev link, you are now required to upload your generated TensorBoard event files directly onto the lab. As an alternative, you may also include a screenshot of your TensorBoard output within your Jupyter notebook. This adjustment ensures that your data visualization and model training efforts are thoroughly documented and accessible for evaluation.

You are also welcome (and encouraged) to utilize alternative logging services like wandB or comet. In such instances, you can easily make your project logs publicly accessible and share the link with others.

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**